In [1]:
from keras import models
from keras.callbacks import EarlyStopping
from keras.layers import Dense, Input
import pandas as pd
import numpy as np
import random

# Данные

In [2]:
LAST_SIGMOID = False
LAST_SIGMOID_K = 6
#COLS_X = ['Поколение', 'Верх/низ', 'Заряд', 'Спин', 'Частица/Античастица']
COLS_X = ['Поколение', 'Верх/низ', 'Электрический заряд', 'Спин', 'Частица/Античастица']
COL_Y = 'Sigmoid массы' if LAST_SIGMOID else 'Логарифм массы'

In [3]:
data = pd.read_csv("Elementary.csv", sep=';', index_col=0)
data

,Название,Поколение,Верх/низ,Электрический заряд,Спин,Масса (МэВ),Логарифм массы,Sigmoid массы,Частица/Античастица,Заряд,Допустимая погрешность (МэВ)
№,,,,,,,,,,,
1,Верхний u-кварк,0.333333,1,0.666667,0.5,2.300000,0.832909,0.568967,1,0.666667,7.000000e-01
2,Нижний d-кварк,0.333333,-1,-0.333333,0.5,4.800000,1.568616,0.627819,1,0.333333,5.000000e-01
3,Очарованный c-кварк,0.666667,1,0.666667,0.5,1275.000000,7.150701,0.915566,1,0.666667,2.500000e+01
4,Странный s-кварк,0.666667,-1,-0.333333,0.5,95.000000,4.553877,0.820238,1,0.333333,5.000000e+00
5,Истинный t-кварк,1.000000,1,0.666667,0.5,174340.000000,12.068763,0.982414,1,0.666667,6.500000e+02
6,Прелестный b-кварк,1.000000,-1,-0.333333,0.5,4180.000000,8.338067,0.941550,1,0.333333,3.000000e+01
7,Верхний u-антикварк,0.333333,1,-0.666667,0.5,2.300000,0.832909,0.568967,-1,0.666667,7.000000e-01
8,Нижний d-антикварк,0.333333,-1,0.333333,0.5,4.800000,1.568616,0.627819,-1,0.333333,5.000000e-01
9,Очарованный c-антикварк,0.666667,1,-0.666667,0.5,1275.000000,7.150701,0.915566,-1,0.666667,2.500000e+01


In [4]:
train_x = data[COLS_X].to_numpy()
train_y = data[COL_Y].to_numpy()
mass = data['Масса (МэВ)'].to_numpy()
abs_error = data['Допустимая погрешность (МэВ)'].to_numpy()

# Модель

In [5]:
# Возвращает абсолютную ошибку
def get_error(real_y, pred_y, is_exp=True):
    return np.abs(real_y - np.exp(pred_y)) if is_exp else np.abs(real_y - pred_y)

In [6]:
model = models.Sequential()
model.add(Dense(6, activation="sigmoid", input_shape=(train_x.shape[1],)))
model.add(Dense(1, activation="linear"))
model.compile(optimizer="adam", loss="mean_squared_error", metrics=["mean_absolute_error"])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 6)                 36        
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 7         
Total params: 43
Trainable params: 43
Non-trainable params: 0
_________________________________________________________________


In [7]:
model.load_weights("model_13_sigmoid_6_1.h5")

In [8]:
pred_error = get_error(mass, model.predict(train_x).ravel())
pred_error

array([7.03121614e-01, 2.26711612e+00, 1.61066162e+02, 5.37914276e+00,
       8.01939062e+04, 6.08781494e+02, 1.15437050e+00, 7.63235283e-01,
       2.25584595e+02, 2.16728668e+02, 1.06144531e+04, 6.10230957e+03,
       2.22833231e-01, 8.30819592e-04, 3.70248003e+00, 3.89343658e-02,
       6.27665840e+02, 1.12654438e+01, 2.47909501e-01, 7.44333834e-04,
       9.97115679e+00, 2.47585633e-02, 5.02129951e+02, 5.01889706e+00,
       3.32911352e+04, 2.76157500e+04, 2.95277148e+04, 2.04570312e+04])

# Обучение

In [9]:
# Возвращает индексы примеров, которые больше заданного порога ошибки
def get_err_examples(train_x, mass, pred_error):
    err = get_error(mass, model.predict(train_x).ravel())
    idx = np.arange(1, train_x.shape[0] + 1)
    return idx[err > pred_error]

# Оставляет ошибку выше заданного порога
def div_err(abs_error, pred_error, koef=2):
    return np.array(list(map(lambda a, b: max(a, b), list(pred_error / koef), list(abs_error))))

In [10]:
idx = get_err_examples(train_x, mass, abs_error)
print(data['Название'].loc[idx].to_list())

['Верхний u-кварк', 'Нижний d-кварк', 'Очарованный c-кварк', 'Странный s-кварк', 'Истинный t-кварк', 'Прелестный b-кварк', 'Верхний u-антикварк', 'Нижний d-антикварк', 'Очарованный c-антикварк', 'Странный s-антикварк', 'Истинный t-антикварк', 'Прелестный b-антикварк', 'Электрон e-', 'Мюон m-', 'Тау-лептон t-', 'Тау-нейтрино vt', 'Позитрон e+', 'Положительный мюон m+', 'Положительный тау-лептон t+', 'Z-бозон', 'W+ бозон', 'W- бозон', 'Бозон Хиггса H']


In [11]:
callback = EarlyStopping(monitor='loss', min_delta=0.0001, patience=3, verbose=0, mode='auto', restore_best_weights=False)

In [12]:
batch = 1
total_max = 10000
for ind in range(10):
    pred_error = div_err(abs_error, pred_error, koef=2)
    bl_idx = True
    jnd = 0
    while bl_idx and jnd < total_max:
        idx = get_err_examples(train_x, mass, pred_error)
        if len(idx) > 0:
            train_x2 = data[COLS_X].loc[idx].to_numpy()
            train_y2 = data[COL_Y].loc[idx].to_numpy()
            results = model.fit(train_x2, train_y2, epochs=3, batch_size=batch, shuffle=False, verbose=0)
            print('{:d} {:} {:}'.format(jnd+1, data['Название'].loc[idx].to_list(), np.mean(results.history["mean_absolute_error"])))
        else:
            pred_error = get_error(mass, model.predict(train_x).ravel())
            bl_idx = False
        jnd += 1
    if jnd >= total_max:
        break

1 ['Верхний u-кварк', 'Нижний d-кварк', 'Очарованный c-кварк', 'Странный s-кварк', 'Истинный t-кварк', 'Прелестный b-кварк', 'Верхний u-антикварк', 'Нижний d-антикварк', 'Очарованный c-антикварк', 'Странный s-антикварк', 'Истинный t-антикварк', 'Прелестный b-антикварк', 'Электрон e-', 'Мюон m-', 'Тау-лептон t-', 'Тау-нейтрино vt', 'Позитрон e+', 'Положительный мюон m+', 'Положительный тау-лептон t+', 'Z-бозон', 'W+ бозон', 'W- бозон', 'Бозон Хиггса H'] 0.2880014677842458
2 ['Нижний d-кварк', 'Очарованный c-кварк', 'Странный s-кварк', 'Верхний u-антикварк', 'Нижний d-антикварк', 'Странный s-антикварк', 'Истинный t-антикварк', 'Прелестный b-антикварк', 'Электрон e-', 'Мюон m-', 'Тау-нейтрино vt', 'Позитрон e+', 'Положительный мюон m+', 'Z-бозон', 'W+ бозон', 'W- бозон', 'Бозон Хиггса H'] 0.304632027943929
3 ['Нижний d-кварк', 'Странный s-кварк', 'Истинный t-кварк', 'Верхний u-антикварк', 'Нижний d-антикварк', 'Истинный t-антикварк', 'Электрон e-', 'Мюон m-', 'Тау-лептон t-', 'Позитрон e+

34 ['Нижний d-кварк', 'Странный s-кварк', 'Прелестный b-кварк', 'Странный s-антикварк', 'Истинный t-антикварк', 'Электронное нейтрино ve', 'Мюон m-', 'Положительный мюон m+'] 0.205232302347819
35 ['Очарованный c-кварк', 'Странный s-кварк', 'Прелестный b-кварк', 'Нижний d-антикварк', 'Очарованный c-антикварк', 'Электрон e-', 'Электронное нейтрино ve', 'Мюон m-', 'Тау-лептон t-', 'Позитрон e+', 'Положительный мюон m+', 'Положительный тау-лептон t+'] 0.1367423137029012
36 ['Нижний d-кварк', 'Странный s-кварк', 'Прелестный b-кварк', 'Нижний d-антикварк', 'Истинный t-антикварк', 'Мюон m-', 'Положительный мюон m+', 'W+ бозон'] 0.16717843214670816
37 ['Очарованный c-кварк', 'Странный s-кварк', 'Прелестный b-кварк', 'Нижний d-антикварк', 'Очарованный c-антикварк', 'Истинный t-антикварк', 'Электрон e-', 'Электронное нейтрино ve', 'Мюон m-', 'Тау-лептон t-', 'Позитрон e+', 'Положительный тау-лептон t+'] 0.11587035655975342
38 ['Нижний d-кварк', 'Странный s-кварк', 'Прелестный b-кварк', 'Странный

79 ['Нижний d-кварк', 'Странный s-кварк', 'Прелестный b-кварк', 'Электронное нейтрино ve', 'Мюон m-', 'Положительный мюон m+'] 0.12516206999619803
80 ['Очарованный c-кварк', 'Странный s-кварк', 'Прелестный b-кварк', 'Электрон e-', 'Электронное нейтрино ve', 'Положительный мюон m+'] 0.11031233767668407
81 ['Нижний d-кварк', 'Странный s-кварк', 'Истинный t-кварк', 'Прелестный b-кварк', 'Истинный t-антикварк', 'Электронное нейтрино ve', 'Мюон m-'] 0.13418545325597128
82 ['Очарованный c-кварк', 'Странный s-кварк', 'Прелестный b-кварк', 'Истинный t-антикварк', 'Электронное нейтрино ve', 'Тау-лептон t-', 'Позитрон e+', 'Положительный мюон m+', 'Положительный тау-лептон t+'] 0.11135848859945933
83 ['Нижний d-кварк', 'Странный s-кварк', 'Прелестный b-кварк', 'Истинный t-антикварк', 'Электронное нейтрино ve', 'Мюон m-', 'Положительный мюон m+'] 0.12963401526212692
84 ['Очарованный c-кварк', 'Странный s-кварк', 'Прелестный b-кварк', 'Очарованный c-антикварк', 'Истинный t-антикварк', 'Электронное

139 ['Нижний d-кварк', 'Странный s-кварк', 'Прелестный b-кварк', 'Электронное нейтрино ve', 'Мюон m-'] 0.13617130617300668
140 ['Странный s-кварк', 'Прелестный b-кварк', 'Очарованный c-антикварк', 'Странный s-антикварк', 'Истинный t-антикварк', 'Электрон e-', 'Электронное нейтрино ve', 'Мюон m-', 'Бозон Хиггса H'] 0.1581815630197525
141 ['Нижний d-кварк', 'Странный s-кварк', 'Истинный t-кварк', 'Прелестный b-кварк', 'Нижний d-антикварк', 'Истинный t-антикварк', 'Мюон m-'] 0.17358233034610748
142 ['Очарованный c-кварк', 'Странный s-кварк', 'Прелестный b-кварк', 'Нижний d-антикварк', 'Очарованный c-антикварк', 'Истинный t-антикварк', 'Электрон e-', 'Электронное нейтрино ve', 'Мюон m-', 'Тау-лептон t-', 'Позитрон e+', 'Положительный мюон m+', 'Положительный тау-лептон t+'] 0.10497584939002991
143 ['Нижний d-кварк', 'Странный s-кварк', 'Прелестный b-кварк', 'Электронное нейтрино ve', 'Мюон m-', 'Положительный мюон m+'] 0.127429169913133
144 ['Странный s-кварк', 'Прелестный b-кварк', 'Очаро

207 ['Нижний d-кварк', 'Странный s-кварк', 'Прелестный b-кварк', 'Истинный t-антикварк', 'Электронное нейтрино ve', 'Мюон m-'] 0.11008328944444656
208 ['Странный s-кварк', 'Прелестный b-кварк', 'Электронное нейтрино ve'] 0.1171415646870931
209 ['Нижний d-кварк', 'Странный s-кварк', 'Прелестный b-кварк', 'Электронное нейтрино ve'] 0.15491739908854166
210 ['Странный s-кварк', 'Прелестный b-кварк', 'Электронное нейтрино ve', 'Мюон m-'] 0.09222277005513509
211 ['Нижний d-кварк', 'Странный s-кварк', 'Прелестный b-кварк', 'Электронное нейтрино ve'] 0.15435942014058432
212 ['Странный s-кварк', 'Прелестный b-кварк', 'Электронное нейтрино ve'] 0.11798545966545741
213 ['Странный s-кварк', 'Прелестный b-кварк', 'Электронное нейтрино ve'] 0.11318932473659515
214 ['Нижний d-кварк', 'Странный s-кварк', 'Прелестный b-кварк', 'Электронное нейтрино ve'] 0.1551701227823893
215 ['Нижний d-кварк', 'Странный s-кварк', 'Прелестный b-кварк', 'Электронное нейтрино ve'] 0.15429596106211343
216 ['Странный s-ква

...

1859 ['Нижний d-кварк', 'Истинный t-кварк', 'Прелестный b-кварк', 'Электрон e-', 'Мюон m-', 'Тау-лептон t-', 'Позитрон e+', 'Положительный мюон m+', 'Мюонное антинейтрино vm', 'Положительный тау-лептон t+', 'Z-бозон', 'W+ бозон', 'W- бозон', 'Бозон Хиггса H'] 0.01380116306245327
1860 ['Странный s-кварк', 'Прелестный b-кварк', 'Прелестный b-антикварк', 'Электрон e-', 'Мюон m-', 'Мюонное нейтрино vm', 'Тау-лептон t-', 'Позитрон e+', 'Положительный мюон m+', 'Мюонное антинейтрино vm', 'Положительный тау-лептон t+', 'Тау-антинейтрино vt', 'Z-бозон', 'W+ бозон', 'W- бозон'] 0.007389701747645934
1861 ['Истинный t-кварк', 'Прелестный b-кварк', 'Странный s-антикварк', 'Истинный t-антикварк', 'Электрон e-', 'Мюон m-', 'Позитрон e+', 'Положительный мюон m+', 'Положительный тау-лептон t+', 'Z-бозон', 'W+ бозон', 'W- бозон', 'Бозон Хиггса H'] 0.005587644952659805
1862 ['Странный s-кварк', 'Истинный t-кварк', 'Прелестный b-антикварк', 'Электрон e-', 'Мюон m-', 'Тау-лептон t-', 'Позитрон e+', 'Полож

1886 ['Истинный t-кварк', 'Прелестный b-кварк', 'Очарованный c-антикварк', 'Истинный t-антикварк', 'Электрон e-', 'Мюон m-', 'Тау-лептон t-', 'Позитрон e+', 'Положительный мюон m+', 'Положительный тау-лептон t+', 'Тау-антинейтрино vt', 'Z-бозон', 'W+ бозон', 'W- бозон', 'Бозон Хиггса H'] 0.02441794363160928
1887 ['Очарованный c-кварк', 'Истинный t-кварк', 'Прелестный b-кварк', 'Очарованный c-антикварк', 'Истинный t-антикварк', 'Прелестный b-антикварк', 'Электрон e-', 'Мюон m-', 'Тау-лептон t-', 'Позитрон e+', 'Положительный мюон m+', 'Положительный тау-лептон t+', 'Z-бозон', 'W- бозон', 'Бозон Хиггса H'] 0.03132512792944908
1888 ['Истинный t-кварк', 'Очарованный c-антикварк', 'Истинный t-антикварк', 'Прелестный b-антикварк', 'Электрон e-', 'Мюон m-', 'Тау-лептон t-', 'Позитрон e+', 'Положительный мюон m+', 'Положительный тау-лептон t+', 'Z-бозон', 'W+ бозон', 'W- бозон', 'Бозон Хиггса H'] 0.032959323997298874
1889 ['Странный s-кварк', 'Истинный t-кварк', 'Истинный t-антикварк', 'Прелес

1913 ['Странный s-кварк', 'Очарованный c-антикварк', 'Истинный t-антикварк', 'Прелестный b-антикварк', 'Электрон e-', 'Мюон m-', 'Мюонное нейтрино vm', 'Тау-лептон t-', 'Позитрон e+', 'Положительный мюон m+', 'Положительный тау-лептон t+', 'Тау-антинейтрино vt', 'Z-бозон', 'W+ бозон', 'W- бозон', 'Бозон Хиггса H'] 0.017787938316663105
1914 ['Истинный t-кварк', 'Прелестный b-кварк', 'Очарованный c-антикварк', 'Истинный t-антикварк', 'Электрон e-', 'Мюон m-', 'Мюонное нейтрино vm', 'Тау-лептон t-', 'Позитрон e+', 'Положительный мюон m+', 'Положительный тау-лептон t+', 'Z-бозон', 'W+ бозон', 'W- бозон', 'Бозон Хиггса H'] 0.03196989186108112
1915 ['Странный s-кварк', 'Истинный t-кварк', 'Прелестный b-кварк', 'Странный s-антикварк', 'Истинный t-антикварк', 'Электрон e-', 'Мюон m-', 'Мюонное нейтрино vm', 'Тау-лептон t-', 'Позитрон e+', 'Положительный мюон m+', 'Мюонное антинейтрино vm', 'Положительный тау-лептон t+', 'Тау-антинейтрино vt', 'Z-бозон', 'W+ бозон', 'W- бозон', 'Бозон Хиггса H'

1955 ['Очарованный c-антикварк', 'Странный s-антикварк', 'Истинный t-антикварк', 'Электрон e-', 'Мюон m-', 'Тау-лептон t-', 'Позитрон e+', 'Положительный мюон m+', 'Положительный тау-лептон t+', 'Тау-антинейтрино vt', 'Z-бозон', 'W+ бозон', 'W- бозон', 'Бозон Хиггса H'] 0.05015423148870468
1956 ['Странный s-кварк', 'Истинный t-кварк', 'Прелестный b-кварк', 'Очарованный c-антикварк', 'Истинный t-антикварк', 'Прелестный b-антикварк', 'Электрон e-', 'Мюон m-', 'Мюонное нейтрино vm', 'Тау-лептон t-', 'Позитрон e+', 'Положительный мюон m+', 'Положительный тау-лептон t+', 'Тау-антинейтрино vt', 'Z-бозон', 'W+ бозон', 'W- бозон', 'Бозон Хиггса H'] 0.03322952054440975
1957 ['Прелестный b-кварк', 'Очарованный c-антикварк', 'Странный s-антикварк', 'Истинный t-антикварк', 'Прелестный b-антикварк', 'Электрон e-', 'Мюон m-', 'Тау-лептон t-', 'Позитрон e+', 'Положительный мюон m+', 'Мюонное антинейтрино vm', 'Положительный тау-лептон t+', 'Тау-антинейтрино vt', 'Z-бозон', 'W+ бозон', 'W- бозон', 'Бо

2006 ['Электрон e-', 'Позитрон e+', 'Положительный мюон m+'] 4.8889053990327135e-05
2007 ['Электрон e-', 'Мюон m-', 'Позитрон e+', 'Z-бозон'] 4.8995018005371094e-05
2008 ['Электрон e-', 'Тау-лептон t-', 'Позитрон e+'] 4.062387718780277e-05
2009 ['Электрон e-', 'Мюон m-', 'Позитрон e+', 'Z-бозон'] 5.3783257802327476e-05
2010 ['Электрон e-', 'Позитрон e+'] 2.863009770711263e-05
2011 ['Электрон e-', 'Позитрон e+', 'Z-бозон'] 3.2385189721632436e-05
2012 ['Электрон e-', 'Позитрон e+', 'Z-бозон'] 1.9285414358212922e-05
2013 ['Позитрон e+'] 7.68899917602539e-06
2014 ['Электрон e-', 'Позитрон e+', 'Z-бозон'] 1.3550122578938803e-05
2015 ['Электрон e-', 'Позитрон e+'] 6.993611653645833e-06
2016 ['Электрон e-', 'Позитрон e+', 'Z-бозон'] 1.2172592884477732e-05
2017 ['Электрон e-', 'Позитрон e+'] 6.9340070088704424e-06
2018 ['Электрон e-', 'Z-бозон'] 1.556674639383952e-05
2019 ['Электрон e-', 'Позитрон e+'] 9.914239247639975e-06
2020 ['Электрон e-', 'Позитрон e+', 'Z-бозон'] 1.1338127857622263e-05


# Предсказание

In [13]:
model.layers[0].get_weights()

[array([[  1.3608266 ,  -4.131684  ,   5.329068  ,  -8.62128   ,
           3.3128247 ,  -2.9637735 ],
        [  3.2097745 ,   2.9238148 ,   2.4312465 ,   6.6908965 ,
           4.6304636 ,   6.270525  ],
        [  9.318507  ,  -0.27915767, -11.6466675 ,  -0.39344582,
         -12.093608  ,   1.5787102 ],
        [ -1.3515481 ,   1.9783188 ,  -2.1837418 ,   0.95869076,
          -1.9780338 ,   1.0719365 ],
        [ -0.16796899,  -0.2485818 ,   0.13774794,  -0.26968023,
          -0.63411605,   1.7198216 ]], dtype=float32),
 array([ 0.10888006, -1.095703  ,  0.53113157,  0.353135  , -0.75451475,
        -2.4775546 ], dtype=float32)]

In [14]:
model.layers[1].get_weights()

[array([[ 23.326588],
        [-20.337273],
        [ 10.07909 ],
        [  6.62472 ],
        [ 13.440715],
        [ -4.773541]], dtype=float32),
 array([-9.631019], dtype=float32)]

In [15]:
pred_y = model.predict(train_x).ravel()
mtrx = np.transpose(np.vstack((pred_y, train_y, pred_y-train_y)))
mtrx

array([[ 8.64941597e-01,  8.32909123e-01,  3.20324740e-02],
       [ 1.48466110e+00,  1.56861592e+00, -8.39548157e-02],
       [ 7.15096760e+00,  7.15070146e+00,  2.66139961e-04],
       [ 4.60038280e+00,  4.55387689e+00,  4.65059129e-02],
       [ 1.20702810e+01,  1.20687627e+01,  1.51833875e-03],
       [ 8.33408260e+00,  8.33806653e+00, -3.98392255e-03],
       [ 9.58579063e-01,  8.32909123e-01,  1.25669940e-01],
       [ 1.51837349e+00,  1.56861592e+00, -5.02424286e-02],
       [ 7.15211773e+00,  7.15070146e+00,  1.41627119e-03],
       [ 4.60215473e+00,  4.55387689e+00,  4.82778398e-02],
       [ 1.20681581e+01,  1.20687627e+01, -6.04540281e-04],
       [ 8.33348751e+00,  8.33806653e+00, -4.57901532e-03],
       [-6.71387672e-01, -6.71387744e-01,  7.15756836e-08],
       [-6.53886986e+00, -6.11929792e+00, -4.19571939e-01],
       [ 4.66014385e+00,  4.66021101e+00, -6.71547661e-05],
       [-1.77203035e+00, -1.77195684e+00, -7.35115461e-05],
       [ 7.48262691e+00,  7.48258053e+00

In [16]:
if LAST_SIGMOID:
    pred_y2 = np.exp(LAST_SIGMOID_K * np.arctanh(2 * pred_y - 1))
    train_y2 = np.exp(LAST_SIGMOID_K * np.arctanh(2 * train_y - 1))
else:
    pred_y2 = np.exp(pred_y)
    train_y2 = np.exp(train_y)
mtrx2 = np.transpose(np.vstack((pred_y2, train_y2, pred_y2-train_y2)))
mtrx2

array([[ 2.37486744e+00,  2.30000000e+00,  7.48674391e-02],
       [ 4.41346931e+00,  4.80000000e+00, -3.86530686e-01],
       [ 1.27533936e+03,  1.27500000e+03,  3.39354949e-01],
       [ 9.95224075e+01,  9.50000000e+01,  4.52240749e+00],
       [ 1.74604906e+05,  1.74339999e+05,  2.64907046e+02],
       [ 4.16338037e+03,  4.18000000e+03, -1.66196309e+01],
       [ 2.60798812e+00,  2.30000000e+00,  3.07988119e-01],
       [ 4.56479454e+00,  4.80000000e+00, -2.35205460e-01],
       [ 1.27680701e+03,  1.27500000e+03,  1.80700632e+00],
       [ 9.96989059e+01,  9.50000000e+01,  4.69890591e+00],
       [ 1.74234641e+05,  1.74339999e+05, -1.05358579e+02],
       [ 4.16090332e+03,  4.18000000e+03, -1.90966817e+01],
       [ 5.10998964e-01,  5.10998950e-01,  1.45268341e-08],
       [ 1.44612195e-03,  2.20000000e-03, -7.53878045e-04],
       [ 1.05651276e+02,  1.05658374e+02, -7.09882651e-03],
       [ 1.69987500e-01,  1.70000000e-01, -1.25002745e-05],
       [ 1.77690247e+03,  1.77682000e+03

# Сохранение

In [17]:
model.save_weights("model_14_sigmoid_6_1.h5")

In [18]:
df = pd.DataFrame(data=mtrx2, columns=['pred', 'real', 'delta'])
df.to_csv('pred.csv', sep=';')